# Initial Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from collections import defaultdict
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
import scipy.stats as stats

# Preprocessing

In [11]:
# Import dataset
df = pd.read_csv("./kieranFeatures_1-30_26-Sep-2024.csv")
n_rows = df.shape[0]

#
# Compute 3-Moving Average for each ID
#
df["avg_adjSA1"] = np.nan
df["avg_adjSA2"] = np.nan
df["avg_adjSA3"] = np.nan
df["avg_adjSAtotal"] = np.nan

cur_row = 1
while cur_row + 1 < n_rows:
    cur_ID = df.loc[cur_row, "ID"]
    next_ID = df.loc[cur_row + 1, "ID"]

    if cur_ID == next_ID:
        # Compute moving average for current trial number if not first or last trial
        df.loc[cur_row, "avg_adjSA1"] = np.mean([df.loc[cur_row - 1, "adjSA1"], df.loc[cur_row, "adjSA1"], df.loc[cur_row + 1, "adjSA1"]])
        df.loc[cur_row, "avg_adjSA2"] = np.mean([df.loc[cur_row - 1, "adjSA2"], df.loc[cur_row, "adjSA2"], df.loc[cur_row + 1, "adjSA1"]])
        df.loc[cur_row, "avg_adjSA3"] = np.mean([df.loc[cur_row - 1, "adjSA3"], df.loc[cur_row, "adjSA3"], df.loc[cur_row + 1, "adjSA1"]])
        df.loc[cur_row, "avg_adjSAtotal"] = np.mean([df.loc[cur_row - 1, "adjSAtotal"], df.loc[cur_row, "adjSAtotal"], df.loc[cur_row + 1, "adjSAtotal"]])

        cur_row += 1
    else:
        # Move to 2nd trial of next participant
        cur_row += 2

# Remove first and last trials of each participant
df.dropna(inplace = True)



#
# Impute Data
#

# Impute missing values with mean of column
df.replace(0, np.nan, inplace = True)
for col in df.columns:
    df[col] = df[col].fillna(value = df[col].mean())



#
# Clear Memory
#
del col, cur_ID, cur_row, n_rows, next_ID

In [ ]:
# Import dataset
df = pd.read_csv("./kieranFeatures_1-31_21-Jan-2025_avgof3.csv").replace("#NAME?", np.nan).replace([np.inf, -np.inf], np.nan)



#
# Impute Data
#

# Impute missing values with mean of column
for col in df.columns:
    df[col] = df[col].fillna(value = df[col].mean())



#
# Split up Dataset
#
ids = df["ID"]
trial_nums = df["trialNum"]
predictors_df = df.drop(columns = ["adjSA1", "adjSA2", ""])

/tmp/ipykernel_2145/272669785.py:2: DtypeWarning: Columns (781,782) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./kieranFeatures_1-31_21-Jan-2025_avgof3.csv").replace("#NAME?", np.nan).replace([np.inf, -np.inf], np.nan)


In [18]:
df.iloc[:, 781]

0          -1.84785
1        252.438729
2          4.902291
3          0.622102
4           0.36862
           ...     
295    -3.402921835
296     5.298826105
297    -3.224659795
298    -2.598417765
299     0.008975581
Name: EEG_frpcv_o1_version11, Length: 300, dtype: object